In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("C://Users//WilliamNayden//Documents//MSDS//case_8.csv", index_col=0)
target = train['target']
train.drop(['target'],inplace=True, axis=1)

In [3]:
train = train.select_dtypes(exclude=['object'])
print(train)

              v1        v2        v4         v5        v6        v7        v8  \
ID                                                                              
3       1.335739  8.727474  3.921026   7.915266  2.599278  3.176895  0.012941   
4       1.630686  7.464411  4.145098   9.191265  2.436402  2.483921  2.301630   
5       0.943877  5.310079  4.410969   5.326159  3.979592  3.928571  0.019645   
6       0.797415  8.304757  4.225930  11.627438  2.097700  1.987549  0.171947   
8       1.630686  7.464411  4.145098   8.742359  2.436402  2.483921  1.496569   
...          ...       ...       ...        ...       ...       ...       ...   
228708  1.630686  7.464411  4.145098   8.742359  2.436402  2.483921  1.496569   
228710  1.630686  7.464411  4.145098   8.742359  2.436402  2.483921  1.496569   
228711  1.630686  7.464411  4.145098  10.069277  2.436402  2.483921  0.323324   
228712  1.630686  7.464411  4.145098  10.106144  2.436402  2.483921  0.309226   
228713  1.619763  7.932978  

In [4]:
print("Dataset has {} entries and {} features".format(*train.shape))

Dataset has 114321 entries and 112 features


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

In [6]:
xgtrain = xgb.DMatrix(X_train.values, y_train.values)
xgtest = xgb.DMatrix(X_test.values, y_test.values)

In [7]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [8]:
from sklearn.metrics import log_loss, accuracy_score

In [9]:
import numpy as np
# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
log_baseline = log_loss(y_test, baseline_predictions)
print("Baseline Log Loss is {:.2f}".format(log_baseline))

Baseline Log Loss is 0.55


In [10]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

In [11]:
params['eval_metric'] = "logloss"

In [12]:
num_boost_round = 999

In [13]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-logloss:0.60271
[1]	Test-logloss:0.55470
[2]	Test-logloss:0.52809
[3]	Test-logloss:0.51300
[4]	Test-logloss:0.50408
[5]	Test-logloss:0.49902
[6]	Test-logloss:0.49553
[7]	Test-logloss:0.49395
[8]	Test-logloss:0.49293
[9]	Test-logloss:0.49266
[10]	Test-logloss:0.49232
[11]	Test-logloss:0.49213
[12]	Test-logloss:0.49227
[13]	Test-logloss:0.49397
[14]	Test-logloss:0.49575
[15]	Test-logloss:0.49584
[16]	Test-logloss:0.49865
[17]	Test-logloss:0.49989
[18]	Test-logloss:0.50006
[19]	Test-logloss:0.50086
[20]	Test-logloss:0.50114
[21]	Test-logloss:0.50195


In [14]:
print("Best Log Loss: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

Best Log Loss: 0.49 with 12 rounds


In [15]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'logloss'},
    early_stopping_rounds=10
)
cv_results

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.601669,0.000238,0.603357,0.000504
1,0.552235,0.000494,0.555576,0.001102
2,0.523861,0.000561,0.528975,0.001601
3,0.506550,0.000594,0.513799,0.001758
4,0.495889,0.000760,0.505380,0.001802
5,0.488966,0.000637,0.500393,0.002177
6,0.484194,0.001207,0.497677,0.002197
7,0.480240,0.001120,0.495854,0.002344
8,0.477260,0.001468,0.495028,0.002489
9,0.474736,0.001477,0.494304,0.002708


In [16]:
cv_results['test-logloss-mean'].min()

0.494134

In [17]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [18]:
# Define initial best params and Log Loss
min_logloss = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )
    # Update best Log Loss
    mean_logloss = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\Log Loss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, Log Loss: {}".format(best_params[0], best_params[1], min_logloss))

CV with max_depth=9, min_child_weight=5
\Log Loss 0.5012734 for 7 rounds
CV with max_depth=9, min_child_weight=6
\Log Loss 0.5012898 for 7 rounds
CV with max_depth=9, min_child_weight=7
\Log Loss 0.5009812 for 7 rounds
CV with max_depth=10, min_child_weight=5
\Log Loss 0.5060884 for 6 rounds
CV with max_depth=10, min_child_weight=6
\Log Loss 0.5052176 for 6 rounds
CV with max_depth=10, min_child_weight=7
\Log Loss 0.5037912 for 6 rounds
CV with max_depth=11, min_child_weight=5
\Log Loss 0.509282 for 5 rounds
CV with max_depth=11, min_child_weight=6
\Log Loss 0.5081260000000001 for 5 rounds
CV with max_depth=11, min_child_weight=7
\Log Loss 0.5071288 for 5 rounds
Best params: 9, 7, Log Loss: 0.5009812


In [19]:
params['max_depth'] = 9
params['min_child_weight'] = 7

In [20]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [22]:
min_logloss = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_logloss = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\tLog Loss {} for {} rounds".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = (subsample,colsample)
print("Best params: {}, {}, Log Loss: {}".format(best_params[0], best_params[1], min_logloss))

CV with subsample=1.0, colsample=1.0
	Log Loss 0.5009812 for 7 rounds
CV with subsample=1.0, colsample=0.9
	Log Loss 0.500625 for 6 rounds
CV with subsample=1.0, colsample=0.8
	Log Loss 0.5006725999999999 for 6 rounds
CV with subsample=1.0, colsample=0.7
	Log Loss 0.5003356 for 7 rounds
CV with subsample=0.9, colsample=1.0
	Log Loss 0.504537 for 6 rounds
CV with subsample=0.9, colsample=0.9
	Log Loss 0.5017894 for 6 rounds
CV with subsample=0.9, colsample=0.8
	Log Loss 0.5025012 for 6 rounds
CV with subsample=0.9, colsample=0.7
	Log Loss 0.5018232 for 6 rounds
CV with subsample=0.8, colsample=1.0
	Log Loss 0.5033380000000001 for 6 rounds
CV with subsample=0.8, colsample=0.9
	Log Loss 0.5030368000000001 for 6 rounds
CV with subsample=0.8, colsample=0.8
	Log Loss 0.5041678 for 5 rounds
CV with subsample=0.8, colsample=0.7
	Log Loss 0.5033358000000001 for 6 rounds
CV with subsample=0.7, colsample=1.0
	Log Loss 0.5064618000000001 for 6 rounds
CV with subsample=0.7, colsample=0.9
	Log Loss 

In [23]:
params['subsample'] = 1.0
params['colsample_bytree'] = 0.7

In [46]:
%time

# This can take some time…
min_logloss = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    
    # We update our parameters
    params['eta'] = eta
    
    # Run and time CV
    %time cv_results = xgb.cv(params,dtrain,num_boost_round=num_boost_round,seed=42,nfold=5,metrics=['logloss'],early_stopping_rounds=10)
    
    # Update best score
    mean_logloss = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\tLog Loss {} for {} rounds\n".format(mean_logloss, boost_rounds))
    if mean_logloss < min_logloss:
        min_logloss = mean_logloss
        best_params = eta
print("Best params: {}, Log Loss: {}".format(best_params, min_logloss))

Wall time: 0 ns
CV with eta=0.3
Wall time: 24.4 s
	Log Loss 0.5003356 for 7 rounds

CV with eta=0.2
Wall time: 36.4 s
	Log Loss 0.49725080000000005 for 12 rounds

CV with eta=0.1
Wall time: 1min 5s
	Log Loss 0.49282200000000004 for 32 rounds

CV with eta=0.05
Wall time: 2min 4s
	Log Loss 0.4909674 for 71 rounds

CV with eta=0.01
Wall time: 12min 44s
	Log Loss 0.48955399999999993 for 439 rounds

CV with eta=0.005
Wall time: 24min 45s
	Log Loss 0.489199 for 894 rounds

Best params: 0.005, Log Loss: 0.489199


In [47]:
params['eta'] = .01

In [48]:
params
{'colsample_bytree': 1.0,
 'eta': 0.01,
 'eval_metric': 'logloss',
 'max_depth': 10,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 0.8}

{'colsample_bytree': 1.0,
 'eta': 0.01,
 'eval_metric': 'logloss',
 'max_depth': 10,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 0.8}

In [50]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best Log Loss: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[0]	Test-logloss:0.68964
[1]	Test-logloss:0.68622
[2]	Test-logloss:0.68283
[3]	Test-logloss:0.67955
[4]	Test-logloss:0.67637
[5]	Test-logloss:0.67319
[6]	Test-logloss:0.67008
[7]	Test-logloss:0.66704
[8]	Test-logloss:0.66429
[9]	Test-logloss:0.66133
[10]	Test-logloss:0.65844
[11]	Test-logloss:0.65563
[12]	Test-logloss:0.65288
[13]	Test-logloss:0.65016
[14]	Test-logloss:0.64750
[15]	Test-logloss:0.64512
[16]	Test-logloss:0.64254
[17]	Test-logloss:0.64001
[18]	Test-logloss:0.63752
[19]	Test-logloss:0.63508
[20]	Test-logloss:0.63268
[21]	Test-logloss:0.63055
[22]	Test-logloss:0.62822
[23]	Test-logloss:0.62593
[24]	Test-logloss:0.62368
[25]	Test-logloss:0.62148
[26]	Test-logloss:0.61950
[27]	Test-logloss:0.61737
[28]	Test-logloss:0.61548
[29]	Test-logloss:0.61342
[30]	Test-logloss:0.61140
[31]	Test-logloss:0.60939
[32]	Test-logloss:0.60764
[33]	Test-logloss:0.60571
[34]	Test-logloss:0.60400
[35]	Test-logloss:0.60213
[36]	Test-logloss:0.60030
[37]	Test-logloss:0.59849
[38]	Test-logloss:0.59

[308]	Test-logloss:0.48915
[309]	Test-logloss:0.48912
[310]	Test-logloss:0.48909
[311]	Test-logloss:0.48907
[312]	Test-logloss:0.48901
[313]	Test-logloss:0.48895
[314]	Test-logloss:0.48892
[315]	Test-logloss:0.48889
[316]	Test-logloss:0.48886
[317]	Test-logloss:0.48883
[318]	Test-logloss:0.48879
[319]	Test-logloss:0.48875
[320]	Test-logloss:0.48871
[321]	Test-logloss:0.48869
[322]	Test-logloss:0.48867
[323]	Test-logloss:0.48863
[324]	Test-logloss:0.48862
[325]	Test-logloss:0.48860
[326]	Test-logloss:0.48857
[327]	Test-logloss:0.48855
[328]	Test-logloss:0.48853
[329]	Test-logloss:0.48848
[330]	Test-logloss:0.48848
[331]	Test-logloss:0.48846
[332]	Test-logloss:0.48842
[333]	Test-logloss:0.48838
[334]	Test-logloss:0.48836
[335]	Test-logloss:0.48834
[336]	Test-logloss:0.48832
[337]	Test-logloss:0.48830
[338]	Test-logloss:0.48827
[339]	Test-logloss:0.48826
[340]	Test-logloss:0.48824
[341]	Test-logloss:0.48820
[342]	Test-logloss:0.48815
[343]	Test-logloss:0.48813
[344]	Test-logloss:0.48812
[